In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
px.defaults.template = "plotly"
import plotly.graph_objects as go

In [3]:
df = pd.read_csv("data.csv")


### GRAPH 1
### Overweight in Time: Boys vs Girls
Story: Nadvaha roste, je to problem, kluci jsou na tom hur nez holky.


In [4]:
### Graph1 
### OVERVIEW OVERWEIGHT IN TIME Boys vs Girls
# Nejdriv zafiltrovat sloupce - az pak dolu
df_filter = df.groupby(["YEAR", "SEX"], as_index=False)["OVERWEIGHT"].mean()
df_filter


,YEAR,SEX,OVERWEIGHT
0,2002,1,0.205935
1,2002,2,0.121896
2,2006,1,0.224768
3,2006,2,0.133559
4,2010,1,0.247421
5,2010,2,0.146501
6,2014,1,0.244559
7,2014,2,0.150621
8,2018,1,0.250991
9,2018,2,0.159768


In [4]:
# Zadani df, columns (pripadne uz zafiltrovane) na osy, nazev grafu

df_filter['SEX'] = df_filter['SEX'].map({2: 'Girls', 1: 'Boys'})
colors = {'Girls': "#eb8fbd", # pink
'Boys': "#3b8ee1"} # blue

fig = px.line(df_filter, y="OVERWEIGHT", x="YEAR", title="Overweight in Time", color="SEX", color_discrete_map=colors)
fig.update_yaxes(range=[0, 0.5])
fig.update_xaxes(tickvals=[2002, 2006, 2010, 2014, 2018])
fig.update_traces(fill="tozeroy")
fig.show()

In [5]:
list_columns = ["SWEETS", "TOOTH_BRUSHING", "BREAKFAST_WEEKDAYS", "BREAKFAST_WEEKEND", "SOFT_DRINKS", "PHYS_ACT_60", "NERVOUS", "FRUITS", "VEGETABLES", "TALK_FATHER","TALK_MOTHER", "FRIEND_TALK", "LIFESAT", "SLEEP_DIF", "TIME_EXE", "FIGHT_YEAR", "HEADACHE", "BUL_BEEN"]
# columns odpovidaji top cca 20 z radom forest modelu + talk_mother pridan jako protipol talk_father


dict_scales = {
    "LIFESAT": 10,
    "FRUITS": 7,
    "SOFT_DRINKS": 7,
    "SWEETS": 7,
    "VEGETABLES": 7,
    "FRIEND_TALK": 7,
    "TIME_EXE": 7,
    "PHYS_ACT_60": 7,
    "DRUNK_30": 7,
    "FAMILY_MEALS_TOGETHER": 6,
    "BREAKFAST_WEEKDAYS": 6,
    "BREAKFAST_WEEKEND": 6,
    "TOOTH_BRUSHING": 5,
    "STUD_TOGETHER": 5,
    "BUL_OTHERS": 5,
    "BUL_BEEN": 5,
    "FIGHT_YEAR": 5,
    "INJURED_YEAR": 5,
    "HEADACHE": 5,
    "STOMACHACHE": 5,
    "FEEL_LOW": 5,
    "NERVOUS": 5,
    "SLEEP_DIF": 5,
    "DIZZY": 5,
    "THINK_BODY": 5,
    "TALK_MOTHER": 5,
    "TALK_FATHER": 5,
    "HEALTH": 4,
    "LIKE_SCHOOL": 4,
    "SCHOOL_PRESSURE": 4,
    "COMPUTER_NO": 4
}


### GRAPH 2
### TOP Gender behaviour differences of overweight children
Story: V jakych factors/types of behaviour se lisi kluci s nadvahou od holek s nadvahou, top chovani ktera jsou nejzajimavejsi/maji nej dopad. 

*How features selected: feature engineering/random forest, regrese a R-squared, kontrolni grafy treba v Tableau

In [7]:
### GRAPH 2
### HORIZONTAL BARCHART ###
# BOYS vs GIRLS
# TOP FEATURES 
# 2018

# narrowing df to year=2018, owerweight=1
df_h = df[(df["YEAR"] == 2018) & (df["OVERWEIGHT"] == 1)]

# specifying 2 groups within the new df - sex: 1 boys, 2 girls
df_h = df_h.groupby("SEX", as_index=False)[list_columns].mean()

# scales normalized to 0-1
for feature in list_columns:
    max_scale = dict_scales[feature]
    df_h[feature] = df_h[feature] / max_scale

# melt = turning df from 'wide' to 'long' form 
df_sex_long = df_h.melt(
    id_vars=["SEX"],                 # keeping this column as it is -> serves as ID of the group
    value_vars=list_columns,         # these columns to be 'melted' into a couple (feature, value):
    var_name="FEATURE",              # new column name for feature
    value_name="VALUE"               # new column name for value
)

# turning numeric 'SEX' (1,2) into text labels
sex_map = {1: "Boys", 2: "Girls"}
df_sex_long["SEX_STRING"] = df_sex_long["SEX"].map(sex_map)     

# výpočet gender gap pro každý faktor (Girls − Boys)
gap_table = (
    df_sex_long
    .groupby(["FEATURE", "SEX_STRING"])["VALUE"]
    .mean()                      # kdyby tam byly duplicity, zprůměruje je
    .unstack("SEX_STRING")
)

gap_table["GIRLS_MINUS_BOYS"] = gap_table["Girls"] - gap_table["Boys"]
gap_table["ABS_GAP"] = gap_table["GIRLS_MINUS_BOYS"].abs()

# TOP 5 faktorů podle absolutní hodnoty gender gapu
top5_features = (
    gap_table
    .sort_values("ABS_GAP", ascending=False)
    .head(20)
    .index
    .tolist()
)

print("TOP 20 faktorů pro graf 2:", top5_features)

# 🔹 Data jen pro TOP 5 faktorů
df_top5 = df_sex_long[df_sex_long["FEATURE"].isin(top5_features)].copy()

# sorting Y features - to be used in category_orders
# pořadí faktorů na ose Y: podle velikosti ABS_GAP (od největšího rozdílu)
feature_order = (
    gap_table.loc[top5_features, "ABS_GAP"]      
    .sort_values(ascending=False)
    .index
    .tolist()
)

# creating BARCHART - GROUPED BARS (NOT STACKED)
colors = {"Girls": "#eb8fbd", "Boys": "#3b8ee1"} 

fig_horizontal_barchart = px.bar(
    df_top5,
    x="VALUE",
    y="FEATURE",
    color="SEX_STRING",
    orientation="h",        # if "v"(vertical) -> then swap x/y axis
    barmode="group",        # group columns next to one another, NOT STACKED
    category_orders={
        "FEATURE": feature_order,
        "SEX_STRING": ["Boys", "Girls"]}, 
    color_discrete_map=colors,
    title="Overweight Children - Boys vs Girls (scaled 0-1, 2018)"
)

fig_horizontal_barchart.update_layout(
    xaxis_title="Average (0-1 scale)",
    yaxis_title="Factor",
    legend_title="Gender"
)

fig_horizontal_barchart.show()


TOP 20 faktorů pro graf 2: ['FIGHT_YEAR', 'TALK_FATHER', 'HEADACHE', 'NERVOUS', 'PHYS_ACT_60', 'TIME_EXE', 'SLEEP_DIF', 'BREAKFAST_WEEKDAYS', 'SOFT_DRINKS', 'TOOTH_BRUSHING', 'FRIEND_TALK', 'LIFESAT', 'VEGETABLES', 'FRUITS', 'TALK_MOTHER', 'SWEETS', 'BREAKFAST_WEEKEND', 'BUL_BEEN']


### GRAPH 3
### Komplexni prehled - Gender differences in all relevant factors
- sirsi obrazek k grafu 2 (tam jen highlights), tady vse


In [31]:
### GRAPH 3_v01
### VERTICAL BARCHART ###
# BOYS vs GIRLS
# other features 
# 2018

df_v = df[(df["YEAR"] == 2018) & (df["OVERWEIGHT"] == 1)]

df_v_group = df_v.groupby("SEX", as_index=False)[list_columns_2].mean()

for feature in list_columns_2:
    max_scale = dict_scales[feature]
    df_v_group[feature] = df_v_group[feature] / max_scale

df_sex_long_2 = df_v_group.melt(
    id_vars=["SEX"],
    value_vars=list_columns_2,
    var_name="FEATURE",
    value_name="VALUE"
)

sex_map = {1: "Boys", 2: "Girls"}
df_sex_long_2["SEX_STRING"] = df_sex_long_2["SEX"].map(sex_map)

feature_order_2 = (
    df_sex_long_2.groupby("FEATURE")["VALUE"]
    .mean()
    .sort_values(ascending=False)
    .index.tolist()
)


# CREATING vertical barchart, not stacked
colors = {"Girls": "#eb8fbd", "Boys": "#3b8ee1"}

fig_vertical = px.bar(
    df_sex_long_2,
    x="FEATURE",                 
    y="VALUE",                   
    color="SEX_STRING",          
    barmode="group",             
    category_orders={
        "FEATURE": feature_order_2,
        "SEX_STRING": ["Boys", "Girls"]  
    },
    color_discrete_map=colors,
    title="Overweight Children - Boys vs Girls (scaled 0-1, 2018)"
)

fig_vertical.update_layout(
    xaxis_title="Factor",
    yaxis_title="Average (0-1 scale)",
    legend_title="Gender"
)

fig_vertical.show()


KeyError: 'BREAKFAST_WEEKEND'

In [16]:
### GRAPH 3_v02
### GENDER GAP - vertical barchart with 2 bars boys/girls
# sorted by gender gap difference DESC
# possibly to replace VIZ3 above

gap_table = (
    df_sex_long_2
    .pivot(index="FEATURE", columns="SEX_STRING", values="VALUE")
    .reset_index()
)

# gender gap: Girls - Boys (+ girls more, - boys more)
gap_table["GAP"] = gap_table["Girls"] - gap_table["Boys"]

# gender gap difference calculated
gap_table["ABS_GAP"] = gap_table["GAP"].abs()

# sorting factors by gender gap DESC
feature_order_2 = (
    gap_table
    .sort_values("ABS_GAP", ascending=False)["FEATURE"]
    .tolist()
)

fig_vertical_gg = px.bar(
    df_sex_long_2,
    x="FEATURE",
    y="VALUE",
    color="SEX_STRING",
    barmode="group",
    category_orders={
        "FEATURE": feature_order_2,
        "SEX_STRING": ["Boys", "Girls"],
    },
    color_discrete_map=colors,
    title="Overweight Children - Boys vs Girls (scaled 0-1, 2018)"
)

fig_vertical_gg.show() 

In [17]:
# GRAPH 3_v03
# GENDER GAP - only one bar - difference girls/boys, sorted DESC
# another option for graph no3

gap_plot_df = gap_table.sort_values("GAP", ascending=False)

# custom barvy: růžová pokud Girls > Boys, modrá pokud Boys > Girls
gap_colors = np.where(gap_plot_df["GAP"] > 0, "#eb8fbd", "#3b8ee1")

fig_gap = px.bar(
    gap_plot_df,
    x="FEATURE",
    y="GAP",
    title="Gender gap by factor (Girls − Boys, overweight children, 2018)",
)

fig_gap.update_traces(marker_color=gap_colors)

fig_gap.update_layout(
    xaxis_title="Factor",
    yaxis_title="Gender gap (Girls − Boys, scaled 0–1)",
    showlegend=False
)

fig_gap.show()

In [ ]:
behavior_factors = ["SWEETS",
                    "TOOTH_BRUSHING", 
                    "BREAKFAST_WEEKDAYS", 
                    "SOFT_DRINKS", 
                    "PHYS_ACT_60", 
                    "NERVOUS", 
                    "FRUITS", 
                    "VEGETABLES", 
                    "TALK_FATHER",
                    "TALK_MOTHER", 
                    "FRIEND_TALK", 
                    "SLEEP_DIF", 
                    "TIME_EXE", 
                    "FIGHT_YEAR", 
                    "HEADACHE", 
                    "BUL_BEEN"
]


sex_map = {1: "Boys", 2: "Girls"}
colors_gender = {"Girls": "#eb8fbd", "Boys": "#3b8ee1"}

# pomocná funkce – normalizace faktorů na 0–1
def normalize_factors(df_in, cols, scales_dict):
    df_norm = df_in.copy()
    for c in cols:
        df_norm[c] = df_norm[c] / scales_dict[c]
    return df_norm

In [19]:
# ==========================================================
# GRAF 2 – Overweight děti 2018 – Boys vs Girls, faktory
# ==========================================================
df_2018 = df[(df["YEAR"] == 2018) & (df["OVERWEIGHT"] == 1)].copy()
df_2018 = normalize_factors(df_2018, behavior_factors, dict_scales)

df_2018_group = (
    df_2018.groupby("SEX", as_index=False)[behavior_factors]
           .mean()
)
df_2018_group["SEX_STRING"] = df_2018_group["SEX"].map(sex_map)

df_long2 = df_2018_group.melt(
    id_vars="SEX_STRING",
    value_vars=behavior_factors,
    var_name="FACTOR",
    value_name="VALUE"
)

# pořadí faktorů podle průměru
factor_order2 = (
    df_long2.groupby("FACTOR")["VALUE"]
            .mean()
            .sort_values(ascending=False)
            .index.tolist()
)

fig2 = px.bar(
    df_long2,
    x="VALUE",
    y="FACTOR",
    color="SEX_STRING",
    orientation="h",
    barmode="group",
    category_orders={"FACTOR": factor_order2,
                     "SEX_STRING": ["Boys", "Girls"]},
    color_discrete_map=colors_gender,
    title="Overweight children 2018 – Boys vs Girls (behaviour factors, scaled 0–1)",
)

fig2.update_layout(
    xaxis_title="Average (0–1 scale)",
    yaxis_title="Factor",
    legend_title="Gender",
)

fig2.show()

In [20]:
# ==========================================================
# GRAF 3 – Gender gap podle faktoru 2018 (Girls − Boys)
# ==========================================================
boys_row = df_2018_group[df_2018_group["SEX_STRING"] == "Boys"].iloc[0]
girls_row = df_2018_group[df_2018_group["SEX_STRING"] == "Girls"].iloc[0]

gaps = []
for f in behavior_factors:
    gap_value = girls_row[f] - boys_row[f]   # Girls − Boys
    gaps.append({"FACTOR": f, "GAP": gap_value})

df_gap = pd.DataFrame(gaps)
df_gap["ABS_GAP"] = df_gap["GAP"].abs()
df_gap = df_gap.sort_values("ABS_GAP", ascending=False)

fig3 = px.bar(
    df_gap,
    x="FACTOR",
    y="GAP",
    title="Gender gap by factor (Girls − Boys, overweight children 2018)",
)

# barvy: růžová pokud Girls > Boys, modrá pokud Boys > Girls
gap_colors = np.where(df_gap["GAP"] > 0, colors_gender["Girls"], colors_gender["Boys"])
fig3.update_traces(marker_color=gap_colors)

fig3.update_layout(
    xaxis_title="Factor",
    yaxis_title="Gap (Girls − Boys, scaled 0–1)",
    showlegend=False,
)

# čísla na sloupcích v %
fig3.update_traces(
    text=df_gap["GAP"],
    texttemplate="%{text:.1%}",
    textposition="outside"
)
fig3.update_yaxes(tickformat=".0%")

fig3.show()

### GRAPH 4
### Behaviour differences: Overweight vs Non-Overweight children

Doplnkovy graf k Kluci vs Holky -> Jak se lisi deti s nadvahou od tech co nemaji nadvahu


In [22]:
# ==========================================================
# NOVÝ GRAF – Overweight vs No-overweight (bez rozdělení na pohlaví)
# ==========================================================

# 1) vezmeme jen rok 2018
df_2018_all = df[df["YEAR"] == 2018].copy()

# 2) normalizace faktorů na 0–1
df_2018_all = normalize_factors(df_2018_all, behavior_factors, dict_scales)

# 3) label pro overweight vs no-overweight
df_2018_all["OW_LABEL"] = np.where(
    df_2018_all["OVERWEIGHT"] == 1,
    "Overweight",
    "No overweight"
)

# 4) průměry faktorů podle OW_LABEL (bez rozdělení na pohlaví)
df_ow_profile = (
    df_2018_all.groupby("OW_LABEL", as_index=False)[behavior_factors]
               .mean()
)

# 5) wide -> long pro Plotly
df_ow_long = df_ow_profile.melt(
    id_vars="OW_LABEL",
    value_vars=behavior_factors,
    var_name="FACTOR",
    value_name="VALUE"
)

# 6) pořadí faktorů podle velikosti rozdílu OW vs No-OW
wide = df_ow_profile.set_index("OW_LABEL")[behavior_factors].T  # řádky = faktory
wide["GAP"] = wide["Overweight"] - wide["No overweight"]
factor_order_ow = (
    wide["GAP"].abs()
        .sort_values(ascending=False)
        .index
        .tolist()
)

# 7) graf – dva sloupce pro každý faktor: No overweight vs Overweight
fig_ow = px.bar(
    df_ow_long,
    x="VALUE",
    y="FACTOR",
    color="OW_LABEL",
    orientation="h",
    barmode="group",
    category_orders={
        "FACTOR": factor_order_ow,
        "OW_LABEL": ["No overweight", "Overweight"]
    },
    title="Behaviour differences: Overweight vs No-overweight children (2018, scaled 0–1)"
)

fig_ow.update_layout(
    xaxis_title="Average (0–1 scale)",
    yaxis_title="Factor",
    legend_title="Group"
)

fig_ow.show()

# Yet another approach
## Overweight / non-overweight


In [6]:
# jen rok 2018
df_2018 = df[df["YEAR"] == 2018].copy()

# (pokud chceš škálu 0–1, odkomentuj):
# df_2018 = normalize_factors(df_2018, behavior_factors, dict_scales)

for feature in list_columns:
    max_scale = dict_scales[feature]
    df_2018[feature] = df_2018[feature] / max_scale

# průměry pro overweight a non-overweight
ow_means  = df_2018[df_2018["OVERWEIGHT"] == 1][list_columns].mean()
non_means = df_2018[df_2018["OVERWEIGHT"] == 0][list_columns].mean()

# rozdíl OW − NON
diff = ow_means.reindex(list_columns) - non_means.reindex(list_columns)


# dataframe pro graf – přes index, aby se nic „neztratilo“
df_diff = diff.reset_index()
df_diff.columns = ["FACTOR", "DIFFERENCE"]

# absolutní rozdíl pro řazení
df_diff["ORIG_IS_NAN"] = df_diff["DIFFERENCE"].isna()
df_diff["DIFFERENCE"] = df_diff["DIFFERENCE"].fillna(0.0)

# seřadit od největšího rozdílu po nejmenší
df_diff["ABS_DIFF"] = df_diff["DIFFERENCE"].abs()
df_diff = df_diff.sort_values("ABS_DIFF", ascending=False)

# label pro barvu
df_diff["SIDE"] = np.where(
    df_diff["DIFFERENCE"] > 0,
    "Overweight higher",
    "Non-overweight higher"
)

color_map = {
    "Overweight higher": "orangered",
    "Non-overweight higher": "seagreen"
}

fig_diff = px.bar(
    df_diff,
    x="DIFFERENCE",
    y="FACTOR",
    orientation="h",
    color="SIDE",
    color_discrete_map=color_map,
    category_orders={"FACTOR": df_diff["FACTOR"].tolist()},
    title="Difference in behaviour (Overweight – Non-overweight), 2018"
)

fig_diff.update_layout(
    xaxis_title="Difference (OW – Non-OW, same scale as input)",
    yaxis_title="Factor",
    legend_title="Group"
)

fig_diff.show()
